In [ ]:
# ==========================================
# CELLULE 1 : MONTAGE GOOGLE DRIVE
# ==========================================

from google.colab import drive
import os

print("="*70)
print("💾 MONTAGE GOOGLE DRIVE")
print("="*70)

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("✅ Google Drive monté")
else:
    print("✅ Google Drive déjà monté")

print("="*70)

💾 MONTAGE GOOGLE DRIVE
Mounted at /content/drive
✅ Google Drive monté


In [ ]:
# ==========================================
# CELLULE 2 : INSTALLATION BARF
# ==========================================

import os
import sys

print("="*70)
print("🔧 INSTALLATION DE L'ENVIRONNEMENT BARF")
print("="*70)

# 1. Créer workspace et cloner BARF
WORKSPACE = '/content/barf_workspace'
BARF_DIR = os.path.join(WORKSPACE, 'bundle-adjusting-NeRF')

os.makedirs(WORKSPACE, exist_ok=True)

if not os.path.exists(BARF_DIR):
    print("\n📥 Clonage du dépôt BARF...")
    os.chdir(WORKSPACE)
    !git clone https://github.com/chenhsuanlin/bundle-adjusting-NeRF.git
    print("✅ Dépôt cloné")
else:
    print("✅ Dépôt BARF déjà présent")

os.chdir(BARF_DIR)
print(f"📂 Répertoire : {os.getcwd()}")

# 2. Installer les dépendances
print("\n📦 Installation des dépendances...")
!pip install -q numpy matplotlib pillow tqdm tensorboard easydict pyyaml opencv-python lpips

# 3. Créer le module SSIM manquant
print("\n🔧 Configuration module SSIM...")
os.makedirs('external', exist_ok=True)

ssim_code = '''"""SSIM implementation"""
import torch
import torch.nn.functional as F
from math import exp

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window

def pytorch_ssim(img1, img2, window_size=11, size_average=True):
    channel = img1.size()[1]
    window = create_window(window_size, channel)
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)

    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding=window_size//2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding=window_size//2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding=window_size//2, groups=channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2
    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)
'''

with open('external/pohsun_ssim.py', 'w') as f:
    f.write(ssim_code)
with open('external/__init__.py', 'w') as f:
    f.write('')

print("✅ Module SSIM créé")

# 4. Patcher options.py pour éviter les input() interactifs
print("\n🔧 Patch de options.py...")

with open('options.py', 'r') as f:
    content = f.read()

# Supprimer les input() interactifs
content = content.replace(
    'override = input("override? (y/n) ")',
    'override = "y"  # Auto-patched for Colab'
)
content = content.replace(
    'retry = input("visdom port ({}) not open, retry? (y/n) ".format(opt.visdom.port))',
    'retry = "n"  # Auto-patched for Colab'
)

with open('options.py', 'w') as f:
    f.write(content)

print("✅ options.py patché")

# 5. Modifier le fichier YAML pour barf_c2f
print("\n📝 Configuration YAML...")

import yaml

yaml_path = 'options/barf_blender.yaml'
with open(yaml_path, 'r') as f:
    config_yaml = yaml.safe_load(f)

config_yaml['barf_c2f'] = [0.1, 0.5]
config_yaml['visdom'] = False

with open(yaml_path, 'w') as f:
    yaml.dump(config_yaml, f, default_flow_style=False)

print("✅ YAML configuré")

print("\n" + "="*70)
print("✅ INSTALLATION TERMINÉE")
print("="*70)

🔧 INSTALLATION DE L'ENVIRONNEMENT BARF

📥 Clonage du dépôt BARF...
Cloning into 'bundle-adjusting-NeRF'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 102 (delta 35), reused 26 (delta 26), pack-reused 46 (from 1)
Receiving objects: 100% (102/102), 191.17 KiB | 6.17 MiB/s, done.
Resolving deltas: 100% (53/53), done.
✅ Dépôt cloné
📂 Répertoire : /content/barf_workspace/bundle-adjusting-NeRF

📦 Installation des dépendances...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00

🔧 Configuration module SSIM...
✅ Module SSIM créé

🔧 Patch de options.py...
✅ options.py patché

📝 Configuration YAML...
✅ YAML configuré

✅ INSTALLATION TERMINÉE


In [ ]:
import os
import subprocess
import re
from datetime import datetime
import pandas as pd

print("="*70)
print("🚀 BARF - ÉVALUATION DE TOUS LES CHECKPOINTS")
print("="*70)

BARF_PATH = '/content/barf_workspace/bundle-adjusting-NeRF'
nerf_path = os.path.join(BARF_PATH, 'model', 'nerf.py')

# ==========================================
# FIX SSIM (une seule fois)
# ==========================================
print("\n🔧 Vérification SSIM...")

with open(nerf_path, 'r') as f:
    lines = f.readlines()

ssim_fixed = False
for i, line in enumerate(lines):
    if 'ssim = pytorch_ssim' in line:
        if 'unsqueeze' not in line:
            indent = ' ' * (len(line) - len(line.lstrip()))
            new_line = indent + 'ssim = pytorch_ssim(rgb_map.permute(0,3,1,2), var.image.permute(0,3,1,2)).item()\n'
            lines[i] = new_line
            ssim_fixed = True

if ssim_fixed:
    with open(nerf_path, 'w') as f:
        f.writelines(lines)
    print("✅ SSIM corrigée")
else:
    print("✅ SSIM déjà correcte")

# ==========================================
# CONFIG
# ==========================================
# Le chemin doit pointer vers le dossier contenant les transforms.json
DATASET_ROOT = '/content/drive/MyDrive/vision/images/vision_scene'

CHECKPOINTS_PATH = '/content/drive/MyDrive/barf_outputs/results/vision_20251207_2007/training/model'
RESULTS_PATH = '/content/drive/MyDrive/barf_outputs/results/evaluation'

os.makedirs(RESULTS_PATH, exist_ok=True)

# ==========================================
# VÉRIFIER LA STRUCTURE DU DATASET
# ==========================================
print("\n🔍 Vérification structure dataset...")
print(f"Dataset root: {DATASET_ROOT}")

# Vérifier si les fichiers transforms existent
transforms_files = [
    os.path.join(DATASET_ROOT, f) for f in os.listdir(DATASET_ROOT)
    if f.startswith('transforms_') and f.endswith('.json')
]

print(f"\nFichiers transforms trouvés:")
for tf in transforms_files:
    print(f"  • {os.path.basename(tf)}")

# Vérifier les chemins dans les fichiers transforms
if transforms_files:
    import json
    with open(transforms_files[0], 'r') as f:
        data = json.load(f)
        if 'frames' in data and len(data['frames']) > 0:
            print(f"\nPremier chemin d'image trouvé: {data['frames'][0].get('file_path', 'Non spécifié')}")

# ==========================================
# CRÉER UN DOSSIER TEMPORAIRE AVEC LA STRUCTURE ATTENDUE PAR BARF
# ==========================================
print("\n🔄 Création structure temporaire pour BARF...")

# Créer un dossier temporaire avec la structure attendue
TEMP_DATASET_ROOT = '/tmp/barf_dataset'
temp_scene_dir = os.path.join(TEMP_DATASET_ROOT, 'vision_scene')
os.makedirs(temp_scene_dir, exist_ok=True)

# Copier les fichiers transforms
for tf in transforms_files:
    import shutil
    dest = os.path.join(temp_scene_dir, os.path.basename(tf))
    shutil.copy2(tf, dest)
    print(f"  • Copié {os.path.basename(tf)} -> {dest}")

# Vérifier et corriger les chemins dans les fichiers transforms
for tf_name in ['transforms_train.json', 'transforms_test.json', 'transforms_val.json']:
    tf_path = os.path.join(temp_scene_dir, tf_name)
    if os.path.exists(tf_path):
        with open(tf_path, 'r') as f:
            data = json.load(f)

        # Corriger les chemins pour qu'ils pointent vers le bon dossier
        frames_updated = False
        for frame in data.get('frames', []):
            if 'file_path' in frame:
                original_path = frame['file_path']
                # Si le chemin commence par "./images/", le corriger
                if original_path.startswith('./images/'):
                    # Garder juste le nom du fichier sans extension
                    img_name = os.path.basename(original_path)
                    # BARF s'attend à des chemins relatifs au dossier de la scène
                    frame['file_path'] = f"./images/{img_name}"
                    frames_updated = True

        if frames_updated:
            with open(tf_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"  • Chemins corrigés dans {tf_name}")

# Copier les images si nécessaire (créer un lien symbolique)
images_src = os.path.join(DATASET_ROOT, 'images')
images_dest = os.path.join(temp_scene_dir, 'images')
if os.path.exists(images_src) and not os.path.exists(images_dest):
    os.symlink(images_src, images_dest)
    print(f"  • Lien symbolique créé: {images_src} -> {images_dest}")

print(f"\n✅ Structure temporaire créée: {temp_scene_dir}")

# ==========================================
# YAML CONFIG
# ==========================================
yaml_path = os.path.join(BARF_PATH, 'options', 'barf_blender_eval.yaml')
yaml_orig = os.path.join(BARF_PATH, 'options', 'barf_blender.yaml')

with open(yaml_orig, 'r') as f:
    yaml = f.read()
yaml = re.sub(r'visdom:\s*(false|False)', 'visdom:\n  cam_depth: 0.5', yaml)
with open(yaml_path, 'w') as f:
    f.write(yaml)

# ==========================================
# LISTE DES CHECKPOINTS
# ==========================================
ckpts = sorted([f for f in os.listdir(CHECKPOINTS_PATH) if f.endswith('.ckpt')])

print(f"\n📂 {len(ckpts)} checkpoints trouvés dans {CHECKPOINTS_PATH}:")
for c in ckpts[:10]:  # Afficher les 10 premiers
    print(f"   • {c}")
if len(ckpts) > 10:
    print(f"   ... et {len(ckpts)-10} autres")

# ==========================================
# ÉVALUATION DE TOUS LES CHECKPOINTS
# ==========================================
all_results = []

print("\n" + "="*70)
print("🎯 DÉBUT DES ÉVALUATIONS")
print("="*70 + "\n")

for idx, ckpt in enumerate(ckpts, 1):
    ckpt_path = os.path.join(CHECKPOINTS_PATH, ckpt)
    iteration = ckpt.replace('.ckpt', '').replace('model_', '')

    print(f"[{idx}/{len(ckpts)}] Évaluation de {ckpt}...")

    # Commande avec la structure temporaire
    cmd = [
        "python", os.path.join(BARF_PATH, "evaluate.py"),
        f"--group=barf_eval_{iteration}",
        "--model=barf",
        "--yaml=barf_blender_eval",
        "--data.scene=vision_scene",
        f"--data.root={TEMP_DATASET_ROOT}",
        f"--load={ckpt_path}",
        "--data.val_on_test=true",
        "--data.dataset=blender"
    ]

    success = False
    metrics = {'checkpoint': ckpt, 'iteration': iteration}

    try:
        print(f"   Exécution de la commande...")
        result = subprocess.run(cmd, capture_output=True, text=True, cwd=BARF_PATH, timeout=600)
        output = result.stdout + result.stderr

        # Afficher les premières et dernières lignes pour debug
        output_lines = output.split('\n')
        print(f"   Return code: {result.returncode}")

        if result.returncode != 0:
            print(f"   ⚠️ Erreurs détectées:")
            for line in output_lines[-20:]:  # Afficher les 20 dernières lignes
                if line.strip():
                    print(f"     {line[:100]}")
        else:
            print(f"   ✅ Commande exécutée avec succès")

            # Sauvegarder le log complet
            log_file = os.path.join(RESULTS_PATH, f"log_{ckpt.replace('.ckpt', '')}.txt")
            with open(log_file, 'w') as f:
                f.write(output)

        # Extraction métriques - chercher le format standard BARF
        for line in output_lines:
            line_lower = line.lower()

            # Format: "PSNR:  24.56" dans les dashboards
            if 'psnr:' in line_lower and not 'psnr_fine' in line_lower:
                matches = re.findall(r'([\d.]+)', line)
                if matches:
                    try:
                        metrics['PSNR'] = float(matches[-1])
                        success = True
                    except:
                        pass

            # Format: "SSIM:  0.8765"
            elif 'ssim:' in line_lower:
                matches = re.findall(r'([\d.]+)', line)
                if matches:
                    try:
                        metrics['SSIM'] = float(matches[-1])
                    except:
                        pass

            # Format: "LPIPS: 0.1234"
            elif 'lpips:' in line_lower:
                matches = re.findall(r'([\d.]+)', line)
                if matches:
                    try:
                        metrics['LPIPS'] = float(matches[-1])
                    except:
                        pass

        # Chercher aussi dans le format de sortie terminal
        for i, line in enumerate(output_lines):
            if '--------------------------' in line:
                # Regarder les 10 lignes après chaque séparateur
                for j in range(i+1, min(i+11, len(output_lines))):
                    content_line = output_lines[j]
                    if 'PSNR:' in content_line and 'PSNR' not in metrics:
                        matches = re.findall(r'([\d.]+)', content_line)
                        if matches:
                            try:
                                metrics['PSNR'] = float(matches[0])
                                success = True
                            except:
                                pass
                    elif 'SSIM:' in content_line and 'SSIM' not in metrics:
                        matches = re.findall(r'([\d.]+)', content_line)
                        if matches:
                            try:
                                metrics['SSIM'] = float(matches[0])
                            except:
                                pass
                    elif 'LPIPS:' in content_line and 'LPIPS' not in metrics:
                        matches = re.findall(r'([\d.]+)', content_line)
                        if matches:
                            try:
                                metrics['LPIPS'] = float(matches[0])
                            except:
                                pass

    except subprocess.TimeoutExpired:
        print(f"   ❌ Timeout (10 minutes)")
        metrics['PSNR'] = None
        metrics['SSIM'] = None
        metrics['LPIPS'] = None
    except Exception as e:
        print(f"   ❌ Erreur d'exécution: {str(e)}")
        metrics['PSNR'] = None
        metrics['SSIM'] = None
        metrics['LPIPS'] = None

    # Vérifier si évaluation a réussi
    if success and 'PSNR' in metrics:
        print(f"   ✅ PSNR: {metrics['PSNR']:.2f} dB | SSIM: {metrics.get('SSIM', 0):.4f} | LPIPS: {metrics.get('LPIPS', 0):.4f}")
        all_results.append(metrics)
    else:
        print(f"   ❌ Échec - métriques non trouvées")
        if 'PSNR' not in metrics:
            metrics['PSNR'] = None
        if 'SSIM' not in metrics:
            metrics['SSIM'] = None
        if 'LPIPS' not in metrics:
            metrics['LPIPS'] = None
        all_results.append(metrics)

        # Afficher la commande pour debug manuel
        print(f"   💡 Commande exécutée:")
        print(f"   {' '.join(cmd)}")

    print()

# ==========================================
# RÉSUMÉ GLOBAL
# ==========================================
print("="*70)
print("📊 RÉSUMÉ GLOBAL")
print("="*70)

if all_results:
    df = pd.DataFrame(all_results)

    # Trier par iteration number
    try:
        df['iter_num'] = pd.to_numeric(df['iteration'], errors='coerce')
        df = df.sort_values('iter_num')
        df = df.drop('iter_num', axis=1)
    except:
        pass

    print("\n" + "="*70)
    print(f"{'Checkpoint':<25} {'Iter':<8} {'PSNR (dB)':<12} {'SSIM':<10} {'LPIPS':<10}")
    print("-"*70)

    for _, row in df.iterrows():
        psnr = f"{row['PSNR']:.2f}" if pd.notna(row['PSNR']) else "N/A"
        ssim = f"{row['SSIM']:.4f}" if pd.notna(row['SSIM']) else "N/A"
        lpips = f"{row['LPIPS']:.4f}" if pd.notna(row['LPIPS']) else "N/A"
        checkpoint_short = row['checkpoint'][:20] + '...' if len(row['checkpoint']) > 23 else row['checkpoint']
        print(f"{checkpoint_short:<25} {row['iteration']:<8} {psnr:<12} {ssim:<10} {lpips:<10}")

    # Statistiques
    valid_results = df[df['PSNR'].notna()].copy()

    if len(valid_results) > 0:
        print("\n" + "="*70)
        print("📈 STATISTIQUES")
        print("-"*70)
        print(f"Checkpoints évalués : {len(valid_results)}/{len(ckpts)}")

        if not valid_results.empty:
            # Meilleur PSNR
            best_psnr_idx = valid_results['PSNR'].idxmax()
            print(f"\nMeilleur PSNR       : {valid_results.loc[best_psnr_idx, 'PSNR']:.2f} dB ({valid_results.loc[best_psnr_idx, 'checkpoint']})")

            # Meilleur SSIM
            if 'SSIM' in valid_results.columns and valid_results['SSIM'].notna().any():
                best_ssim_idx = valid_results['SSIM'].idxmax()
                print(f"Meilleur SSIM       : {valid_results.loc[best_ssim_idx, 'SSIM']:.4f} ({valid_results.loc[best_ssim_idx, 'checkpoint']})")

            # Meilleur LPIPS
            if 'LPIPS' in valid_results.columns and valid_results['LPIPS'].notna().any():
                best_lpips_idx = valid_results['LPIPS'].idxmin()
                print(f"Meilleur LPIPS      : {valid_results.loc[best_lpips_idx, 'LPIPS']:.4f} ({valid_results.loc[best_lpips_idx, 'checkpoint']})")

            print(f"\nMoyenne PSNR        : {valid_results['PSNR'].mean():.2f} dB")
            if 'SSIM' in valid_results.columns and valid_results['SSIM'].notna().any():
                print(f"Moyenne SSIM        : {valid_results['SSIM'].mean():.4f}")
            if 'LPIPS' in valid_results.columns and valid_results['LPIPS'].notna().any():
                print(f"Moyenne LPIPS       : {valid_results['LPIPS'].mean():.4f}")

    # Sauvegarder CSV
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_path = os.path.join(RESULTS_PATH, f"all_checkpoints_{timestamp}.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n💾 CSV sauvegardé: {csv_path}")

    # Graphique évolution (texte ASCII)
    if len(valid_results) > 1 and 'PSNR' in valid_results.columns:
        print("\n" + "="*70)
        print("📈 ÉVOLUTION DU PSNR")
        print("-"*70)

        psnr_values = valid_results['PSNR'].values
        min_psnr = psnr_values.min()
        max_psnr = psnr_values.max()

        for _, row in valid_results.iterrows():
            psnr = row['PSNR']
            # Normaliser entre 0 et 50 caractères
            if pd.notna(psnr) and max_psnr > min_psnr:
                bar_len = int(((psnr - min_psnr) / (max_psnr - min_psnr)) * 50)
            else:
                bar_len = 25
            bar = '█' * bar_len
            checkpoint_short = row['checkpoint'][:12] + '...' if len(row['checkpoint']) > 15 else row['checkpoint']
            print(f"{checkpoint_short:<15} {bar} {psnr:.2f} dB")

else:
    print("\n❌ Aucun résultat obtenu")

# Rapport détaillé
report_path = os.path.join(RESULTS_PATH, f"rapport_complet_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
with open(report_path, 'w') as f:
    f.write("="*70 + "\n")
    f.write("RAPPORT D'ÉVALUATION COMPLÈTE - TOUS LES CHECKPOINTS\n")
    f.write("="*70 + "\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Nombre de checkpoints: {len(ckpts)}\n")
    f.write(f"Checkpoints path: {CHECKPOINTS_PATH}\n")
    f.write(f"Dataset original: {DATASET_ROOT}\n")
    f.write(f"Dataset temporaire: {TEMP_DATASET_ROOT}\n")
    f.write(f"Scene name: vision_scene\n\n")

    f.write("STRUCTURE DATASET:\n")
    f.write("-"*70 + "\n")
    f.write(f"Transforms files:\n")
    for tf in transforms_files:
        f.write(f"  • {os.path.basename(tf)}\n")

    f.write("\n" + "="*70 + "\n")
    f.write("RÉSULTATS DÉTAILLÉS:\n")
    f.write("-"*70 + "\n")
    if all_results:
        for r in all_results:
            f.write(f"\nCheckpoint: {r['checkpoint']}\n")
            f.write(f"Iteration: {r['iteration']}\n")
            f.write(f"  PSNR  : {r.get('PSNR', 'N/A')}\n")
            f.write(f"  SSIM  : {r.get('SSIM', 'N/A')}\n")
            f.write(f"  LPIPS : {r.get('LPIPS', 'N/A')}\n")
    else:
        f.write("\nAucun résultat disponible\n")

print(f"\n💾 Rapport détaillé: {report_path}")

# Nettoyage (optionnel)
import shutil
if os.path.exists(TEMP_DATASET_ROOT):
    shutil.rmtree(TEMP_DATASET_ROOT)
    print(f"🧹 Structure temporaire nettoyée: {TEMP_DATASET_ROOT}")

print("\n" + "="*70)
print("✅ ÉVALUATION COMPLÈTE TERMINÉE")
print("="*70)

🚀 BARF - ÉVALUATION DE TOUS LES CHECKPOINTS

🔧 Vérification SSIM...
✅ SSIM corrigée

🔍 Vérification structure dataset...
Dataset root: /content/drive/MyDrive/vision/images/vision_scene

Fichiers transforms trouvés:
  • transforms_train.json
  • transforms_test.json
  • transforms_val.json

Premier chemin d'image trouvé: ./images/img_76

🔄 Création structure temporaire pour BARF...
  • Copié transforms_train.json -> /tmp/barf_dataset/vision_scene/transforms_train.json
  • Copié transforms_test.json -> /tmp/barf_dataset/vision_scene/transforms_test.json
  • Copié transforms_val.json -> /tmp/barf_dataset/vision_scene/transforms_val.json
  • Chemins corrigés dans transforms_train.json
  • Chemins corrigés dans transforms_test.json
  • Chemins corrigés dans transforms_val.json

✅ Structure temporaire créée: /tmp/barf_dataset/vision_scene

📂 2 checkpoints trouvés dans /content/drive/MyDrive/barf_outputs/results/vision_20251207_2007/training/model:
   • 78000.ckpt
   • 80000.ckpt

🎯 DÉBUT DES